In [40]:
import pandas as pd
import requests
import os
import warnings
warnings.filterwarnings("ignore")

storage_path = "../../data/raw/kba/"
os.makedirs(storage_path, exist_ok=True)

base_url = "https://www.kba.de/SharedDocs/Downloads/DE/Statistik/Fahrzeuge/FZ10/"
report_type = "fz10"

year_months = [
                
               "2025_10", "2025_09", "2025_08", "2025_07", "2025_06", 
               "2025_05", "2025_04", "2025_03", "2025_02", "2025_01",

               "2024_12", "2024_11", "2024_10", "2024_09", "2024_08", 
               "2024_07", "2024_06", "2024_05", "2024_04", "2024_03", 
               "2024_02", "2024_01"
               
               ]

# Download Excel File
for year_month in year_months:
    file_path = os.path.join(storage_path, f"{report_type}_{year_month}.xlsx")
    
    # Check if file already exists to avoid re-downloading
    if os.path.exists(file_path):
        print(f"File already exists: {file_path}")
        continue
        
    print("Downloading data for", year_month)
    url = f"{base_url}{report_type}_{year_month}.xlsx?__blob=publicationFile&v=2"

    response = requests.get(url)

    # Save the file locally
    with open(file_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved to {file_path}")


File already exists: ../../data/raw/kba/fz10_2025_10.xlsx
File already exists: ../../data/raw/kba/fz10_2025_09.xlsx
File already exists: ../../data/raw/kba/fz10_2025_08.xlsx
File already exists: ../../data/raw/kba/fz10_2025_07.xlsx
File already exists: ../../data/raw/kba/fz10_2025_06.xlsx
File already exists: ../../data/raw/kba/fz10_2025_05.xlsx
File already exists: ../../data/raw/kba/fz10_2025_04.xlsx
File already exists: ../../data/raw/kba/fz10_2025_03.xlsx
File already exists: ../../data/raw/kba/fz10_2025_02.xlsx
File already exists: ../../data/raw/kba/fz10_2025_01.xlsx
File downloaded and saved to ../../data/raw/kba/fz10_2024_12.xlsx
File downloaded and saved to ../../data/raw/kba/fz10_2024_11.xlsx
File downloaded and saved to ../../data/raw/kba/fz10_2024_11.xlsx
File downloaded and saved to ../../data/raw/kba/fz10_2024_10.xlsx
File downloaded and saved to ../../data/raw/kba/fz10_2024_09.xlsx
File downloaded and saved to ../../data/raw/kba/fz10_2024_10.xlsx
File downloaded and save

In [41]:
# List all donwnloaded files
available_files = os.listdir(storage_path)
print("Downloaded files:", available_files)

Downloaded files: ['fz10_2025_02.xlsx', 'fz10_2024_03.xlsx', 'fz10_2024_02.xlsx', 'fz10_2025_03.xlsx', 'fz10_2025_08.xlsx', 'fz10_2024_09.xlsx', 'fz10_2025_04.xlsx', 'fz10_2024_05.xlsx', 'fz10_2024_12.xlsx', 'fz10_2024_04.xlsx', 'fz10_2025_05.xlsx', 'fz10_2024_08.xlsx', 'fz10_2025_09.xlsx', 'fz10_2024_11.xlsx', 'fz10_2024_07.xlsx', 'fz10_2025_06.xlsx', 'fz10_2025_10.xlsx', 'fz10_2025_07.xlsx', 'fz10_2024_06.xlsx', 'fz10_2024_10.xlsx', 'fz10_2024_01.xlsx', 'fz10_2025_01.xlsx']


In [42]:
def data_cleaning(file_path: str, 
                  sheet_name: str = "FZ 10.1", 
                  skiprows: int = 7 
                  ) -> pd.DataFrame:
    """Cleans the KBA Excel file and returns a cleaned DataFrame."""

    # Rename German columns to short English versions
    column_mapping = {
        'Insgesamt': 'Total',
        'mit Dieselantrieb': 'Diesel',
        'mit Hybridantrieb': 'Hybrid_column',
        'mit Hybridantrieb \n(incl. Plug-in-Hybrid)': 'Hybrid_All',
        'Benzin-Hybridantrieb \n(incl. Plug-in-Hybrid)': 'Hybrid_Petrol_All',
        'Diesel-Hybridantrieb \n(incl. Plug-in-Hybrid)': 'Hybrid_Diesel_All',
        'Hybridantrieb \n(ohne Plug-in-Hybrid)': 'Hybrid_NonPlugin',
        'Benzin-Hybridantrieb \n(ohne Plug-in-Hybrid)': 'Hybrid_Petrol_NonPlugin',
        'Diesel-Hybridantrieb \n(ohne Plug-in-Hybrid)': 'Hybrid_Diesel_NonPlugin',
        'Plug-in-Hybridantrieb': 'Hybrid_Plugin',
        'Benzin-Plug-in-Hybridantrieb': 'Hybrid_Petrol_Plugin',
        'Diesel-Plug-in-Hybridantrieb': 'Hybrid_Diesel_Plugin',
        'mit Elektroantrieb (BEV)': 'Electric_BEV',
        'mit Allradantrieb': 'All_Wheel_Drive',
        'Cabriolets': 'Convertibles'
    }

    year, month = file_path.split("_")[1], file_path.split("_")[-1].split(".")[0]
    year_month_date = pd.to_datetime(f"{year}-{month}-01") + pd.offsets.MonthEnd(0)

    # Load the Excel file
    df = pd.read_excel(file_path, 
                       sheet_name=sheet_name, 
                       skiprows=skiprows)

    # Rename columns
    df.rename(columns={"Unnamed: 1": "OEM", 
                    "Unnamed: 2": "Model"}, inplace=True)

    # Filter all values after value "NEUZULASSUNGEN INSGESAMT"
    index_of_last_row = df[df['OEM'].str.contains('NEUZULASSUNGEN INSGESAMT').fillna(False)].index[0]
    df = df.iloc[1:index_of_last_row]

    # Select only columns with absolute numbers
    selected_columns = [x for x in df.columns if not 'Unnamed' in x]
    df = df[selected_columns].copy()

    # Filter out rows where OEM is 'ZUSAMMEN'
    df = df[~df['OEM'].str.contains('ZUSAMMEN', na=False)]

    # Fill out missing values in 'OEM' column
    df['OEM'].ffill(inplace=True)

    # Fill out missing values "-" with 0 
    df.replace("-", 0, inplace=True)

    # Validate no missing values remain
    try:
        assert df.isnull().any().any() == False
    except AssertionError:
        print("Warning: There are still missing values in the DataFrame for file ", file_path)
        print(df.isnull().sum())

    df.rename(columns=column_mapping, inplace=True)

    # All columns containing the word 'Hybrid' are added up
    hybrid_columns = [col for col in df.columns if 'Hybrid' in col]
    df['Hybrid'] = df[hybrid_columns].sum(axis=1)
    df.drop(columns=hybrid_columns, inplace=True)

    # Calculate 'Petrol' columns as Total - Diesel - Electric - Hybrid
    df['Petrol'] = df['Total'] - df['Diesel'] - df['Electric_BEV'] - df['Hybrid']

    # Reshape DataFrame from wide to long format
    df = df.melt(id_vars=["OEM", "Model"], 
        var_name="drive_type", 
        value_name="Value")
    
    df["Date"] = year_month_date

    df['ts_key'] = df['OEM'] +  "_" + df['Model'] + "_" + df['drive_type']

    # sort values 
    df.sort_values(by=['Date', 'ts_key'], inplace=True)

    return df

In [43]:
dfs = []

#for file_name in available_files:
for file_name in ['fz10_2024_03.xlsx']:

    file_path = os.path.join(storage_path, file_name)
    
    try:
        df_cleaned = data_cleaning(file_path)
    except Exception as e:
        print(e) 
    
    dfs.append(df_cleaned)
    
    del df_cleaned

# Join all dataframes
df = pd.concat(dfs)

OEM                                              0
Model                                            3
Insgesamt                                        0
mit Dieselantrieb                                0
mit Hybridantrieb \n(incl. Plug-in-Hybrid)       0
Benzin-Hybridantrieb \n(incl. Plug-in-Hybrid)    0
Diesel-Hybridantrieb \n(incl. Plug-in-Hybrid)    0
Hybridantrieb \n(ohne Plug-in-Hybrid)            0
Benzin-Hybridantrieb \n(ohne Plug-in-Hybrid)     0
Diesel-Hybridantrieb \n(ohne Plug-in-Hybrid)     0
Plug-in-Hybridantrieb                            0
Benzin-Plug-in-Hybridantrieb                     0
Diesel-Plug-in-Hybridantrieb                     0
mit Elektroantrieb (BEV)                         0
mit Allradantrieb                                0
Cabriolets                                       0
dtype: int64
unsupported operand type(s) for +: 'int' and 'str'


NameError: name 'df_cleaned' is not defined

In [36]:
df.head()

,OEM,Model,drive_type,Value,Date,ts_key
1200,ALFA ROMEO,GIULIA,All_Wheel_Drive,57,2025-02-28,ALFA ROMEO_GIULIA_All_Wheel_Drive
1600,ALFA ROMEO,GIULIA,Convertibles,0,2025-02-28,ALFA ROMEO_GIULIA_Convertibles
400,ALFA ROMEO,GIULIA,Diesel,4,2025-02-28,ALFA ROMEO_GIULIA_Diesel
800,ALFA ROMEO,GIULIA,Electric_BEV,0,2025-02-28,ALFA ROMEO_GIULIA_Electric_BEV
2000,ALFA ROMEO,GIULIA,Hybrid,0,2025-02-28,ALFA ROMEO_GIULIA_Hybrid


In [37]:
df['OEM'].unique()

array(['ALFA ROMEO', 'ALPINE', 'ASTON MARTIN', 'AUDI', 'BENTLEY', 'BMW',
       'BYD', 'CADILLAC', 'CITROEN', 'DACIA', 'DS', 'FERRARI', 'FIAT',
       'FISKER', 'FORD', 'GWM', 'HONDA', 'HYUNDAI', 'INEOS', 'IVECO',
       'JAGUAR', 'JEEP', 'KIA', 'LAMBORGHINI', 'LANCIA', 'LAND ROVER',
       'LEAPMOTOR', 'LEXUS', 'LOTUS', 'LUCID', 'LYNK & CO', 'MAN',
       'MASERATI', 'MAXUS', 'MAZDA', 'MERCEDES', 'MG ROEWE', 'MINI',
       'MITSUBISHI', 'MORGAN', 'NIO', 'NISSAN', 'OPEL', 'PEUGEOT',
       'POLESTAR', 'PORSCHE', 'RENAULT', 'ROLLS ROYCE', 'SEAT', 'SKODA',
       'SMART', 'SSANGYONG', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA',
       'VINFAST', 'VOLVO', 'VW', 'XPENG', 'CHERY', 'DAF TRUCKS', 'KGM',
       'LADA', 'TOGG', 'DEEPAL', 'OMODA'], dtype=object)

In [38]:
df[df['ts_key'] == 'ALFA ROMEO_GIULIA_Total'].sort_values(by=['Date'])

,OEM,Model,drive_type,Value,Date,ts_key
0,ALFA ROMEO,GIULIA,Total,94,2025-01-31,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,78,2025-02-28,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,95,2025-03-31,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,78,2025-04-30,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,107,2025-05-31,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,93,2025-06-30,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,74,2025-07-31,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,85,2025-08-31,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,46,2025-09-30,ALFA ROMEO_GIULIA_Total
0,ALFA ROMEO,GIULIA,Total,55,2025-10-31,ALFA ROMEO_GIULIA_Total


In [39]:
df[df['ts_key'] == 'MERCEDES_A-KLASSE_Total']

,OEM,Model,drive_type,Value,Date,ts_key
205,MERCEDES,A-KLASSE,Total,1257,2025-02-28,MERCEDES_A-KLASSE_Total
211,MERCEDES,A-KLASSE,Total,1629,2025-03-31,MERCEDES_A-KLASSE_Total
242,MERCEDES,A-KLASSE,Total,1354,2025-08-31,MERCEDES_A-KLASSE_Total
223,MERCEDES,A-KLASSE,Total,1480,2025-04-30,MERCEDES_A-KLASSE_Total
229,MERCEDES,A-KLASSE,Total,1413,2025-05-31,MERCEDES_A-KLASSE_Total
248,MERCEDES,A-KLASSE,Total,2062,2025-09-30,MERCEDES_A-KLASSE_Total
234,MERCEDES,A-KLASSE,Total,1549,2025-06-30,MERCEDES_A-KLASSE_Total
251,MERCEDES,A-KLASSE,Total,2454,2025-10-31,MERCEDES_A-KLASSE_Total
238,MERCEDES,A-KLASSE,Total,1615,2025-07-31,MERCEDES_A-KLASSE_Total
190,MERCEDES,A-KLASSE,Total,645,2025-01-31,MERCEDES_A-KLASSE_Total
